#### Import Modules

In [5]:
%load_ext autoreload
%autoreload 2

In [25]:
###### import os
import pandas as pd
from glob import glob 
from dotenv import load_dotenv
import matplotlib.pyplot as plt
environment_loaded = load_dotenv()

#### Instantiate dask optimisations

In [78]:
import itertools

In [60]:
d = {
    'GPS': 
        [{'_id': 1, 'action': 'Long', 'units': 20, 'date': '2022-02-03', 'price': 14.57, 'simulatedFee': 0}
        ,{'_id': 2, 'action': 'Short', 'units': 20, 'date': '2022-04-04', 'price': 14.21, 'simulatedFee': 0}
        ,{'_id': 3, 'action': 'Long', 'units': 25, 'date': '2022-06-13', 'price': 9.03, 'simulatedFee': 0.2}],
    'IAU': 
        [{'_id': 1, 'action': 'Long', 'units': 10, 'date': '2022-03-02', 'price': 36.61, 'simulatedFee': 0}
        ,{'_id': 2, 'action': 'Short', 'units': 10, 'date': '2022-03-08', 'price': 38.98, 'simulatedFee': 0}
        ,{'_id': 3, 'action': 'Long', 'units': 5, 'date': '2022-03-15', 'price': 36.45, 'simulatedFee': 0.1}
        ,{'_id': 4, 'action': 'Long', 'units': 5, 'date': '2022-04-13', 'price': 37.62, 'simulatedFee': 0.23}],
    'M': 
        [{'_id': 1, 'action': 'Long', 'units': 55, 'date': '2022-05-24', 'price': 17.61, 'simulatedFee': 0}],
    'NFLX': 
        [{'_id': 1, 'action': 'Long', 'units': 2, 'date': '2022-01-19', 'price': 515.86, 'simulatedFee': 0}],
    'STNG': 
        [{'_id': 1, 'action': 'Long', 'units': 50, 'date': '2022-04-24', 'price': 20.60, 'simulatedFee': 0.26}
        ,{'_id': 2, 'action': 'Long', 'units': 20, 'date': '2022-05-13', 'price': 28.35, 'simulatedFee': 0.29}
        ,{'_id': 3, 'action': 'Short', 'units': 45, 'date': '2022-05-24', 'price': 30.46, 'simulatedFee': 0.31}
        ,{'_id': 4, 'action': 'Long', 'units': 20, 'date': '2022-06-02', 'price': 35.19, 'simulatedFee': 0.35}]
}

In [83]:
%%timeit
cd = []
for k, v in d.items():
    cd += [[k] * len(v)]

tickers = [*itertools.chain.from_iterable(cd)]
holdings = [*itertools.chain.from_iterable(d.values())]

3.16 µs ± 48.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [100]:
%%timeit
pd.DataFrame(holdings, index = tickers).sort_values("date").reset_index()

1.99 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [101]:
%%timeit
pd.DataFrame(holdings, index = tickers).sort_values("date")

1.31 ms ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [105]:
%%timeit
sorted(zip(holdings, tickers), key=lambda d: d[0]['date']) 

3.41 µs ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Initiate mulithreading 

In [3]:
from typing import Callable, Dict, Any, List
from api.gcp.connect_cloud import CloudUtility
from concurrent.futures import ThreadPoolExecutor

In [4]:
cloud_util = CloudUtility()

In [6]:
from experiments.parallelizers import parallelize_factory
ticker_data = cloud_util.read_file_from_gcs("tickers/historical_ticks_5/")

### Static Analysis

#### Instantiate sample data

In [26]:
s = {
    'GPS': 
        [{'_id': 1, 'action': 'Long', 'units': 20, 'date': '', 'price': 14.57, 'simulatedFee': 0}],
    'IAU': 
        [{'_id': 1, 'action': 'Long', 'units': 10, 'date': '', 'price': 36.61, 'simulatedFee': 0}],
    'M': 
        [{'_id': 1, 'action': 'Long', 'units': 55, 'date': '', 'price': 17.61, 'simulatedFee': 0}],
    'NFLX': 
        [{'_id': 1, 'action': 'Long', 'units': 2, 'date': '', 'price': 515.86, 'simulatedFee': 0}],
    'STNG': 
        [{'_id': 1, 'action': 'Long', 'units': 50, 'date': '', 'price': 20.60, 'simulatedFee': 0.26}]
}

In [27]:
data = {
    'GPS': np.random.random(9600) * 10 + 50,
    'IAU': np.random.random(9600) * 10 + 15,
    'M': np.random.random(9600) * 10 + 10,
    'NFLX': np.random.random(9600) * 10 + 200,
    'STNG': np.random.random(9600) * 10 + 20,
}
units = np.fromiter([*map(lambda x: x[0]['units'],[*s.values()])], dtype = float)

##### Get Portfolio Value over time

In [37]:
ts_arr = np.array([*data.values()])
pf_raw_value_ot = ts_arr.T.dot(units)
pf_raw_value_ot

array([3920.9433998 , 3844.9954384 , 3771.47127365, ..., 3746.77136326,
       3395.43195632, 3517.15751896])

#### Alternative Indicators

##### Get Correlation Matrix of tickers

In [42]:
from experiments.correlations import spearman_corr_matrix, rolling_spearman_corr
spearman_corr_matrix(ts_arr)

array([[ 1.00000000e+00, -1.08769099e-02,  4.12434206e-03,
         8.23005011e-03,  9.66479844e-03],
       [-1.08769099e-02,  1.00000000e+00, -8.19873250e-03,
        -2.03547801e-02,  4.64328224e-03],
       [ 4.12434206e-03, -8.19873250e-03,  1.00000000e+00,
        -1.02580297e-02,  9.56054671e-04],
       [ 8.23005011e-03, -2.03547801e-02, -1.02580297e-02,
         1.00000000e+00, -1.14179070e-02],
       [ 9.66479844e-03,  4.64328224e-03,  9.56054671e-04,
        -1.14179070e-02,  1.00000000e+00]])

##### Get Market Correlation over time

In [59]:
market_values = np.random.random(9600) * 10 * 300
rolling_spearman_corr(market_values, pf_raw_value_ot, window = 28)

array([        nan,         nan,         nan, ...,  0.0831965 ,
       -0.01258894, -0.03174603])

### Dynamic Portfolio Metrics

In [ ]:
from typing import Iterable

def days_open(date_series: Iterable) -> np.uint16:
    return np.uint16((date_series.max() - date_series.min()).days)

def turnover_rate(date_series: Iterable) -> np.float64:
    date_series = np.fromiter(date_series, dtype='datetime64[D]')
    days_traded = days_open(date_series)
    return np.round(date_series.size / days_traded, 4)